# Projeto 3

<b>Tema:</b> Estimar o retorno financeiro de um filme

<b>Técnica:</b> Regressão Linear

<b>Grupo:</b> Lucas Leal Vale, Matheus Augusto Soares, Rafael Almada
___

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
movies=pd.read_csv('movies_metadata.csv', low_memory=False)
imdb=pd.read_csv('ratings_small.csv',low_memory=False)

In [3]:
movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [4]:
df=movies.loc[:, {'id','original_title','original_language','genres','popularity','runtime','budget','revenue'}]
df.head()

,revenue,id,popularity,runtime,genres,budget,original_language,original_title
0,373554033.0,862,21.946943,81.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",30000000,en,Toy Story
1,262797249.0,8844,17.015539,104.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",65000000,en,Jumanji
2,0.0,15602,11.7129,101.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",0,en,Grumpier Old Men
3,81452156.0,31357,3.859495,127.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",16000000,en,Waiting to Exhale
4,76578911.0,11862,8.387519,106.0,"[{'id': 35, 'name': 'Comedy'}]",0,en,Father of the Bride Part II


In [5]:
#Tirando budgets diferente de 0 (budget ainda como string
df=df[df.budget != "0"]
df=df.drop([19730, 29503,35587])

In [6]:
#Transformando em float id e budget
df.id=df.id.astype('float64')
df.budget=df.budget.astype('float64')
df.dtypes

revenue              float64
id                   float64
popularity            object
runtime              float64
genres                object
budget               float64
original_language     object
original_title        object
dtype: object

In [7]:
#Ordenando em ordem crescente de acordo com o id, e filtrando valores 0 das variáveis float
df=df[(df.revenue >0) & (df.budget>0)  & (df.runtime>0)]
df=df.sort_values(['id'], ascending=[True])
df.head()

,revenue,id,popularity,runtime,genres,budget,original_language,original_title
17,4300000.0,5.0,9.026586,98.0,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",4000000.0,en,Four Rooms
256,775398007.0,11.0,42.149697,121.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",11000000.0,en,Star Wars
6232,940335536.0,12.0,25.497794,100.0,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",94000000.0,en,Finding Nemo
351,677945399.0,13.0,48.307194,142.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",55000000.0,en,Forrest Gump
2742,356296601.0,14.0,20.726578,122.0,"[{'id': 18, 'name': 'Drama'}]",15000000.0,en,American Beauty


In [8]:
#Tirando media dos ratings
imdb=imdb.sort_values(['movieId'], ascending=[True])
MediaRating=imdb.drop("userId",1,inplace=True)
MediaRating=imdb.drop("timestamp",1,inplace=True)
imdb.columns

Index(['movieId', 'rating'], dtype='object')

In [9]:
imdb.head()

,movieId,rating
9713,1,4.0
35933,1,1.5
52631,1,5.0
35983,1,2.5
12038,1,4.0


In [18]:
rating=pd.Series(imdb.groupby(['movieId'])['rating'].mean())

In [19]:
rating

movieId
1         3.872470
2         3.401869
3         3.161017
4         2.384615
5         3.267857
6         3.884615
7         3.283019
8         3.800000
9         3.150000
10        3.450820
11        3.689024
12        2.861111
13        3.937500
14        3.451613
15        2.318182
16        3.948864
17        3.924419
18        3.288462
19        2.597826
20        2.538462
21        3.536842
22        3.355263
23        3.090909
24        3.044118
25        3.742574
26        4.100000
27        3.142857
28        4.083333
29        4.025000
30        4.050000
            ...   
158238    3.750000
158314    4.500000
158528    3.500000
158956    4.000000
159093    2.000000
159462    3.000000
159690    2.000000
159755    1.000000
159858    3.750000
159972    0.500000
160080    1.000000
160271    2.500000
160438    4.250000
160440    1.500000
160563    2.500000
160565    2.000000
160567    4.000000
160590    5.000000
160656    3.500000
160718    4.000000
161084    2.500000
1611

In [16]:
imdb

,movieId,rating
9713,1,4.0
35933,1,1.5
52631,1,5.0
35983,1,2.5
12038,1,4.0
72180,1,4.0
85228,1,4.0
72201,1,3.0
84897,1,1.5
84775,1,3.0
